In [ ]:
# 有两个作者的文章（A, B），定义为0， 1
A = 0 # hamilton
B = 1 # madison
UNKNOWN = -1

def preprocessing(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
        text = ' '.join(lines[1:]).replace('\n',' ').replace('  ', ' ').lower().replace('hamilton','').replace('madison','')
        text = ' '.join(text.split())
        return text

    # 把同一作者的文章全部合并到一个文件
    textA, textB = '', ''

    import os
    for file in os.listdir('./papers/A'):
        textA += preprocessing('./papers/A/'+file)
    for file in os.listdir('./papers/B'):
        textB += preprocessing('./papers/B/'+file)
